<a href="https://colab.research.google.com/github/Aryan130103/Data-Science-Projects/blob/main/Deepfake_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import dlib
import librosa
from sklearn.model_selection import train_test_split
import seaborn as sns
from tqdm import tqdm

# Mount Google Drive to access the CelebDF dataset
drive.mount('/content/drive')

# Define paths
celeb_df_path = '/content/drive/MyDrive/Colab Notebooks/archive'  # Update with your actual path
output_path = '/content/processed_data'

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Step 1: Dataset Exploration and Preprocessing
def explore_dataset(dataset_path):
    """Explore the structure and contents of the CelebDF dataset"""
    real_videos = []
    fake_videos = []

    # Assuming dataset has 'real' and 'fake' directories
    for root, dirs, files in os.walk(dataset_path):
        if 'real' in root.lower():
            for file in files:
                if file.endswith(('.mp4', '.avi')):
                    real_videos.append(os.path.join(root, file))
        elif 'fake' in root.lower():
            for file in files:
                if file.endswith(('.mp4', '.avi')):
                    fake_videos.append(os.path.join(root, file))

    print(f"Found {len(real_videos)} real videos and {len(fake_videos)} fake videos")

    # Sample a few videos to analyze
    sample_real = real_videos[:5] if real_videos else []
    sample_fake = fake_videos[:5] if fake_videos else []

    video_stats = []

    # Analyze sample videos
    for video_path in sample_real + sample_fake:
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()

        video_stats.append({
            'path': video_path,
            'type': 'real' if 'real' in video_path.lower() else 'fake',
            'fps': fps,
            'frame_count': frame_count,
            'duration': duration,
            'resolution': f"{width}x{height}"
        })

    stats_df = pd.DataFrame(video_stats)
    print("\nVideo Statistics:")
    print(stats_df.describe())

    return real_videos, fake_videos, stats_df

# Step 2: Face Detection and Extraction
def extract_faces(video_path, output_folder, max_frames=30):
    """Extract faces from video frames using dlib"""
    # Create face detector
    detector = dlib.get_frontal_face_detector()

    # Create output folder
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_output = os.path.join(output_folder, video_name)
    os.makedirs(video_output, exist_ok=True)

    # Open video
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate sampling interval to evenly distribute frame selection
    if total_frames <= max_frames:
        frame_indices = range(total_frames)
    else:
        frame_indices = np.linspace(0, total_frames-1, max_frames, dtype=int)

    face_count = 0
    frame_metrics = []

    # Process frames
    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()

        if not ret:
            continue

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = detector(gray)

        # Process each face in the frame
        for i, face in enumerate(faces):
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()

            # Add margin to face bounding box
            margin = 30
            x1 = max(0, x1 - margin)
            y1 = max(0, y1 - margin)
            x2 = min(frame.shape[1], x2 + margin)
            y2 = min(frame.shape[0], y2 + margin)

            # Extract face ROI
            face_roi = frame[y1:y2, x1:x2]

            # Only save if the face has reasonable dimensions
            if face_roi.shape[0] > 50 and face_roi.shape[1] > 50:
                face_filename = f"{video_output}/face_{frame_idx:04d}_{i}.jpg"
                cv2.imwrite(face_filename, face_roi)
                face_count += 1

                # Calculate basic metrics about the face
                face_height, face_width = face_roi.shape[:2]
                face_area = face_height * face_width

                frame_metrics.append({
                    'video': video_name,
                    'frame': frame_idx,
                    'face_id': i,
                    'face_width': face_width,
                    'face_height': face_height,
                    'face_area': face_area,
                    'bbox': (x1, y1, x2, y2)
                })

    cap.release()

    return face_count, frame_metrics

# Step 3: Facial Landmark Detection
def extract_facial_landmarks(image_path):
    """Extract facial landmarks using dlib's 68-point predictor"""
    # Load the detector and predictor
    detector = dlib.get_frontal_face_detector()

    # This path needs to be updated - predictor file needs to be downloaded
    predictor_path = "/content/shape_predictor_68_face_landmarks.dat"

    # Download the shape predictor if it doesn't exist
    if not os.path.exists(predictor_path):
        !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
        !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

    predictor = dlib.shape_predictor(predictor_path)

    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        return None, None

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    if not faces:
        return img, None

    # Get the largest face based on area
    largest_face = max(faces, key=lambda rect: rect.width() * rect.height())

    # Get facial landmarks
    landmarks = predictor(gray, largest_face)

    # Convert landmarks to numpy array
    points = np.zeros((68, 2), dtype=int)
    for i in range(68):
        points[i] = (landmarks.part(i).x, landmarks.part(i).y)

    # Draw landmarks on the image for visualization
    for (x, y) in points:
        cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

    return img, points

# Step 4: Feature Extraction - Multi-modal
def extract_features_from_face(image_path):
    """Extract comprehensive features from facial image"""
    img = cv2.imread(image_path)
    if img is None:
        return None

    features = {}

    # 1. Basic image statistics
    features['mean_color'] = img.mean(axis=(0, 1)).tolist()
    features['std_color'] = img.std(axis=(0, 1)).tolist()

    # 2. Face-specific metrics from facial landmarks
    _, landmarks = extract_facial_landmarks(image_path)

    if landmarks is not None:
        # Eye aspect ratio (EAR) - can detect blinking inconsistencies
        def eye_aspect_ratio(eye_points):
            # Compute the euclidean distances between sets of points
            A = np.linalg.norm(eye_points[1] - eye_points[5])
            B = np.linalg.norm(eye_points[2] - eye_points[4])
            C = np.linalg.norm(eye_points[0] - eye_points[3])
            # Return the eye aspect ratio
            return (A + B) / (2.0 * C)

        # Left eye points (36-41 in dlib's 68-point model)
        left_eye = landmarks[36:42]
        # Right eye points (42-47 in dlib's 68-point model)
        right_eye = landmarks[42:48]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        features['left_eye_aspect_ratio'] = left_ear
        features['right_eye_aspect_ratio'] = right_ear

        # Mouth aspect ratio - can detect lip sync issues
        mouth_top = landmarks[51]
        mouth_bottom = landmarks[57]
        mouth_left = landmarks[48]
        mouth_right = landmarks[54]

        mouth_width = np.linalg.norm(mouth_left - mouth_right)
        mouth_height = np.linalg.norm(mouth_top - mouth_bottom)

        features['mouth_aspect_ratio'] = mouth_height / mouth_width if mouth_width > 0 else 0

        # Symmetry score - comparing left and right side of face
        # This is crucial for deepfake detection as many deepfakes have subtle asymmetry
        left_eye_center = left_eye.mean(axis=0)
        right_eye_center = right_eye.mean(axis=0)

        nose_tip = landmarks[30]
        face_midpoint = (left_eye_center + right_eye_center) / 2

        # Distance between nose and face midpoint (ideally small for real faces)
        features['nose_midpoint_offset'] = np.linalg.norm(nose_tip - face_midpoint)

        # Check if jaw is symmetrical
        jaw_left = landmarks[:8]  # Left side of jaw
        jaw_right = landmarks[8:17]  # Right side of jaw

        # Flip jaw_right horizontally to compare with jaw_left
        face_midline_x = face_midpoint[0]
        jaw_right_flipped = np.array([(2*face_midline_x - p[0], p[1]) for p in jaw_right])

        # Compute average distance between corresponding points
        jaw_asymmetry = np.mean([np.linalg.norm(p1 - p2) for p1, p2 in zip(jaw_left, jaw_right_flipped[::-1])])
        features['jaw_asymmetry'] = jaw_asymmetry

    # 3. Image quality assessment - deepfakes often have compression artifacts
    # Laplacian variance - measure of image blur
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    features['laplacian_variance'] = laplacian_var

    # 4. Color consistency checks - helps detect lighting inconsistencies
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    features['hsv_mean'] = hsv.mean(axis=(0, 1)).tolist()
    features['hsv_std'] = hsv.std(axis=(0, 1)).tolist()

    return features

# Step 5: Audio Feature Extraction
def extract_audio_features(video_path, output_folder):
    """Extract audio features from video"""
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    audio_path = os.path.join(output_folder, f"{video_name}_audio.wav")

    # Extract audio using ffmpeg
    os.system(f"ffmpeg -i '{video_path}' -q:a 0 -map a '{audio_path}' -y")

    # Check if audio extraction was successful
    if not os.path.exists(audio_path) or os.path.getsize(audio_path) < 1000:
        return None

    # Load audio file
    try:
        y, sr = librosa.load(audio_path, sr=None)
    except:
        return None

    features = {}

    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    features['mfcc_mean'] = np.mean(mfccs, axis=1).tolist()
    features['mfcc_std'] = np.std(mfccs, axis=1).tolist()

    # Extract spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    features['spectral_centroid_mean'] = np.mean(spectral_centroids)
    features['spectral_centroid_std'] = np.std(spectral_centroids)

    # Extract zero crossing rate
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    features['zcr_mean'] = np.mean(zcr)
    features['zcr_std'] = np.std(zcr)

    # Extract tempo (beats per minute)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    features['tempo'] = tempo

    # Remove the temporary audio file
    os.remove(audio_path)

    return features

# Main Execution Function for Phase 1
def execute_phase1(celeb_df_path, output_path):
    """Execute all steps of Phase 1"""
    # Step 1: Dataset Exploration
    print("Step 1: Exploring dataset...")
    real_videos, fake_videos, stats_df = explore_dataset(celeb_df_path)

    # Create processing directories
    real_faces_dir = os.path.join(output_path, 'real_faces')
    fake_faces_dir = os.path.join(output_path, 'fake_faces')
    os.makedirs(real_faces_dir, exist_ok=True)
    os.makedirs(fake_faces_dir, exist_ok=True)

    # Visualize dataset statistics
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    sns.histplot(stats_df['fps'], kde=True)
    plt.title('FPS Distribution')

    plt.subplot(2, 2, 2)
    sns.histplot(stats_df['duration'], kde=True)
    plt.title('Duration Distribution (seconds)')

    plt.subplot(2, 2, 3)
    stats_df['resolution_width'] = stats_df['resolution'].apply(lambda x: int(x.split('x')[0]))
    stats_df['resolution_height'] = stats_df['resolution'].apply(lambda x: int(x.split('x')[1]))
    plt.scatter(stats_df['resolution_width'], stats_df['resolution_height'], c=['blue' if t=='real' else 'red' for t in stats_df['type']])
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.title('Video Resolutions (Blue=Real, Red=Fake)')

    plt.subplot(2, 2, 4)
    sns.countplot(x='type', data=stats_df)
    plt.title('Real vs Fake Videos')

    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'dataset_stats.png'))

    # Step 2 & 3: Process a subset of videos for development
    # Using a smaller subset for initial development
    real_subset = real_videos[:50] if len(real_videos) > 50 else real_videos
    fake_subset = fake_videos[:50] if len(fake_videos) > 50 else fake_videos

    all_face_metrics = []
    all_features = []

    print("\nStep 2: Extracting faces from videos...")
    for video_path in tqdm(real_subset + fake_subset):
        is_real = 'real' in video_path.lower()
        output_dir = real_faces_dir if is_real else fake_faces_dir

        # Extract faces from video
        face_count, metrics = extract_faces(video_path, output_dir)

        if metrics:
            for metric in metrics:
                metric['is_real'] = is_real
                all_face_metrics.append(metric)

    # Create a DataFrame with face metrics
    face_metrics_df = pd.DataFrame(all_face_metrics)
    face_metrics_df.to_csv(os.path.join(output_path, 'face_metrics.csv'), index=False)

    # Visualize face extraction results
    print(f"Total faces extracted: {len(face_metrics_df)}")

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    sns.histplot(data=face_metrics_df, x='face_width', hue='is_real', kde=True, common_norm=False)
    plt.title('Face Width Distribution')

    plt.subplot(2, 2, 2)
    sns.histplot(data=face_metrics_df, x='face_height', hue='is_real', kde=True, common_norm=False)
    plt.title('Face Height Distribution')

    plt.subplot(2, 2, 3)
    sns.histplot(data=face_metrics_df, x='face_area', hue='is_real', kde=True, common_norm=False)
    plt.title('Face Area Distribution')

    plt.subplot(2, 2, 4)
    sns.countplot(data=face_metrics_df, x='is_real')
    plt.title('Real vs Fake Faces')

    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'face_extraction_stats.png'))

    # Step 4 & 5: Extract features from faces and audio
    print("\nStep 4 & 5: Extracting facial and audio features...")

    # Get all extracted face images
    all_face_images = []
    for root, _, files in os.walk(real_faces_dir):
        for file in files:
            if file.endswith('.jpg'):
                all_face_images.append((os.path.join(root, file), True))

    for root, _, files in os.walk(fake_faces_dir):
        for file in files:
            if file.endswith('.jpg'):
                all_face_images.append((os.path.join(root, file), False))

    # Process a subset of face images to extract features
    face_subset = all_face_images[:1000] if len(all_face_images) > 1000 else all_face_images

    for face_path, is_real in tqdm(face_subset):
        # Extract visual features
        visual_features = extract_features_from_face(face_path)

        if visual_features:
            visual_features['is_real'] = is_real
            visual_features['image_path'] = face_path

            # Get original video path from face path
            video_name = os.path.basename(os.path.dirname(face_path))
            video_path = None

            if is_real and real_videos:
                for v in real_videos:
                    if video_name in v:
                        video_path = v
                        break
            elif not is_real and fake_videos:
                for v in fake_videos:
                    if video_name in v:
                        video_path = v
                        break

            # Extract audio features if video path is found
            if video_path:
                audio_features = extract_audio_features(video_path, output_path)
                if audio_features:
                    visual_features.update({f"audio_{k}": v for k, v in audio_features.items()})

            all_features.append(visual_features)

    # Create features DataFrame
    if all_features:
        # Convert list of dictionaries to DataFrame
        features_df = pd.DataFrame([{k: str(v) if isinstance(v, list) else v
                                  for k, v in d.items()}
                                 for d in all_features])

        # Save features to CSV
        features_df.to_csv(os.path.join(output_path, 'extracted_features.csv'), index=False)

        print(f"Saved {len(features_df)} feature records to CSV")
    else:
        print("No features were extracted")

    # Return the paths to the processed data
    return {
        'real_faces_dir': real_faces_dir,
        'fake_faces_dir': fake_faces_dir,
        'face_metrics_file': os.path.join(output_path, 'face_metrics.csv'),
        'features_file': os.path.join(output_path, 'extracted_features.csv'),
        'dataset_stats_image': os.path.join(output_path, 'dataset_stats.png'),
        'face_stats_image': os.path.join(output_path, 'face_extraction_stats.png')
    }

# Execute Phase 1
results = execute_phase1(celeb_df_path=celeb_df_path, output_path=output_path)
print("Phase 1 completed successfully!")
print(f"Results saved to: {output_path}")

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import random
from tqdm import tqdm
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from einops import rearrange
import warnings
warnings.filterwarnings('ignore')

# Phase 2: Model Development & Adversarial Training
print("Starting Phase 2: Model Development & Adversarial Training")

# Define paths from Phase 1
output_path = '/content/processed_data'
real_faces_dir = os.path.join(output_path, 'real_faces')
fake_faces_dir = os.path.join(output_path, 'fake_faces')

# Step 1: Create PyTorch Dataset for Deepfake Detection
class DeepfakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None, max_samples=5000, temporal_frames=8):
        self.transform = transform
        self.temporal_frames = temporal_frames

        # Get all image paths
        real_images = self._get_image_paths(real_dir)
        fake_images = self._get_image_paths(fake_dir)

        # Limit dataset size if needed
        real_images = real_images[:max_samples] if len(real_images) > max_samples else real_images
        fake_images = fake_images[:max_samples] if len(fake_images) > max_samples else fake_images

        # Group frames by video for temporal analysis
        self.real_videos = self._group_by_video(real_images)
        self.fake_videos = self._group_by_video(fake_images)

        # Create image-level samples for spatial model
        self.images = [(img, 1) for img in real_images] + [(img, 0) for img in fake_images]

        # Create video-level samples for temporal model
        self.videos = []
        for video_frames in self.real_videos.values():
            if len(video_frames) >= temporal_frames:
                self.videos.append((video_frames, 1))

        for video_frames in self.fake_videos.values():
            if len(video_frames) >= temporal_frames:
                self.videos.append((video_frames, 0))

        print(f"Loaded {len(self.images)} total images, {len(self.videos)} video sequences")
        print(f"Real videos: {len(self.real_videos)}, Fake videos: {len(self.fake_videos)}")

    def _get_image_paths(self, directory):
        return glob(os.path.join(directory, "**/*.jpg"), recursive=True)

    def _group_by_video(self, image_paths):
        videos = {}
        for img_path in image_paths:
            video_name = os.path.basename(os.path.dirname(img_path))
            if video_name not in videos:
                videos[video_name] = []
            videos[video_name].append(img_path)
        return videos

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # For spatial analysis (single frame)
        if hasattr(self, 'mode') and self.mode == 'temporal':
            video_frames, label = self.videos[idx % len(self.videos)]

            # Randomly select consecutive frames
            if len(video_frames) <= self.temporal_frames:
                selected_frames = video_frames
            else:
                start_idx = random.randint(0, len(video_frames) - self.temporal_frames)
                selected_frames = video_frames[start_idx:start_idx + self.temporal_frames]

            # Load and transform frames
            frames = []
            for frame_path in selected_frames:
                frame = cv2.imread(frame_path)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                if self.transform:
                    frame = self.transform(image=frame)["image"]
                frames.append(frame)

            # Stack frames into tensor
            frames_tensor = np.stack(frames, axis=0)
            return frames_tensor, label

        # Default: spatial mode (single image)
        img_path, label = self.images[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)["image"]

        return image, label

    def set_mode(self, mode):
        """Switch between spatial and temporal mode"""
        self.mode = mode
        if mode == 'spatial':
            print(f"Dataset in spatial mode with {len(self.images)} samples")
        else:
            print(f"Dataset in temporal mode with {len(self.videos)} samples")

# Step 2: Create transformations & augmentations
def get_transforms(mode='train'):
    if mode == 'train':
        return A.Compose([
            A.Resize(224, 224),
            A.HorizontalFlip(p=0.5),
            A.OneOf([
                A.GaussianBlur(p=0.5),
                A.MotionBlur(p=0.5),
            ], p=0.3),
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, p=0.3),
            A.Normalize(),
            A.pytorch.transforms.ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(),
            A.pytorch.transforms.ToTensorV2(),
        ])

# Step 3: Create Vision Transformer Model for Spatial Analysis
class ViTForDeepfake(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        # Load pretrained ViT
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=pretrained)

        # Replace classifier head
        self.vit.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.vit.head.in_features, 1)
        )

    def forward(self, x):
        x = self.vit(x)
        return x

# Step 4: Create 3D CNN Model for Temporal Analysis
class VideoTransformer(nn.Module):
    def __init__(self, num_frames=8, pretrained=True):
        super().__init__()
        # Spatial feature extractor (ResNet backbone)
        self.backbone = timm.create_model('resnet18', pretrained=pretrained, features_only=True)

        # Get feature dimension
        dummy = torch.zeros(1, 3, 224, 224)
        features = self.backbone(dummy)
        feature_dim = features[-1].shape[1]

        # Temporal Transformer
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=feature_dim,
                nhead=8,
                dim_feedforward=2048,
                dropout=0.1,
                batch_first=True
            ),
            num_layers=2
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.LayerNorm(feature_dim),
            nn.Linear(feature_dim, 1)
        )

    def forward(self, x):
        b, t, c, h, w = x.shape
        # Reshape for backbone processing
        x = rearrange(x, 'b t c h w -> (b t) c h w')

        # Extract spatial features
        features = self.backbone(x)
        # Use the deepest features
        x = features[-1]

        # Global average pooling
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(-1, x.size(1))

        # Reshape back to batch, time, features
        x = rearrange(x, '(b t) c -> b t c', b=b, t=t)

        # Apply temporal transformer
        x = self.temporal_transformer(x)

        # Global temporal pooling
        x = x.mean(dim=1)

        # Classification
        x = self.classifier(x)
        return x

# Step 5: Adversarial Sample Generation
class GAN(nn.Module):
    def __init__(self):
        super().__init__()
        # Generator
        self.generator = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return x + 0.1 * self.generator(x)  # Residual connection for small perturbations

# Step 6: Training Functions
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs=5):
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in pbar:
            inputs = inputs.to(device)
            labels = labels.float().to(device).view(-1, 1)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward + optimize
            loss.backward()
            optimizer.step()

            # Statistics
            train_loss += loss.item() * inputs.size(0)
            pred = torch.sigmoid(outputs) > 0.5
            train_correct += (pred == labels).sum().item()
            train_total += labels.size(0)

            pbar.set_postfix({'loss': loss.item(), 'acc': train_correct / train_total})

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.float().to(device).view(-1, 1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                pred = torch.sigmoid(outputs) > 0.5
                val_correct += (pred == labels).sum().item()
                val_total += labels.size(0)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total

        # Update learning rate
        if scheduler:
            scheduler.step(val_loss)

        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, val_loss={val_loss:.4f}, val_acc={val_acc:.4f}')

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), os.path.join(output_path, 'best_model.pth'))

    return history

# Step 7: Adversarial Training
def adversarial_training(detection_model, adv_model, train_loader, criterion, optimizer, device, alpha=0.5):
    detection_model.train()
    adv_model.train()

    adv_optimizer = optim.Adam(adv_model.parameters(), lr=0.0002)

    for inputs, labels in tqdm(train_loader, desc=f'Adversarial Training'):
        inputs = inputs.to(device)
        labels = labels.float().to(device).view(-1, 1)

        # Generate adversarial samples
        adv_optimizer.zero_grad()
        adv_inputs = adv_model(inputs)

        # Train generator to fool detector
        adv_outputs = detection_model(adv_inputs)
        # Generator wants to make real images classified as fake
        real_indices = (labels == 1).squeeze()
        if real_indices.sum() > 0:
            adv_loss = criterion(adv_outputs[real_indices], torch.zeros_like(labels[real_indices]))
            adv_loss.backward()
            adv_optimizer.step()

        # Train detector on both original and adversarial samples
        optimizer.zero_grad()

        # Original samples
        outputs = detection_model(inputs)
        loss_orig = criterion(outputs, labels)

        # Adversarial samples (detached)
        with torch.no_grad():
            adv_inputs = adv_model(inputs).detach()

        adv_outputs = detection_model(adv_inputs)
        # For real samples made to look fake, target is still real
        loss_adv = criterion(adv_outputs, labels)

        # Combined loss
        loss = (1-alpha) * loss_orig + alpha * loss_adv
        loss.backward()
        optimizer.step()

# Step 8: Main training function
def execute_phase2(real_faces_dir, fake_faces_dir, output_path):
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create datasets
    train_transform = get_transforms(mode='train')
    val_transform = get_transforms(mode='val')

    # Create dataset
    dataset = DeepfakeDataset(real_faces_dir, fake_faces_dir, transform=train_transform)

    # Split dataset
    train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)

    # Create subsets
    from torch.utils.data import Subset
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

    print(f"Training on {len(train_dataset)} samples, validating on {len(val_dataset)} samples")

    # Step 1: Train ViT Model for Spatial Analysis
    print("\nTraining Vision Transformer for Spatial Analysis...")
    vit_model = ViTForDeepfake().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(vit_model.parameters(), lr=2e-5, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # Train for fewer epochs to showcase the code
    vit_history = train_model(
        vit_model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs=3
    )

    # Step 2: Switch to temporal mode and train Video Transformer
    print("\nTraining Video Transformer for Temporal Analysis...")
    dataset.set_mode('temporal')

    # Create new dataloaders
    train_temporal_dataset = Subset(dataset, train_indices[:len(train_indices)//2])  # Use a subset for speed
    val_temporal_dataset = Subset(dataset, val_indices[:len(val_indices)//2])

    train_temporal_loader = DataLoader(train_temporal_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_temporal_loader = DataLoader(val_temporal_dataset, batch_size=8, shuffle=False, num_workers=4)

    # Create and train model
    video_model = VideoTransformer().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(video_model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # Train for fewer epochs to showcase the code
    video_history = train_model(
        video_model, train_temporal_loader, val_temporal_loader, criterion, optimizer, scheduler, device, num_epochs=2
    )

    # Step 3: Create adversarial training setup
    print("\nPerforming Adversarial Training...")
    dataset.set_mode('spatial')  # Switch back to spatial mode

    # Create adversarial model
    adv_model = GAN().to(device)

    # Train with adversarial samples
    # Reset optimizer for new training phase
    optimizer = optim.AdamW(vit_model.parameters(), lr=1e-5, weight_decay=1e-4)

    # Do one epoch of adversarial training
    adversarial_training(vit_model, adv_model, train_loader, criterion, optimizer, device)

    # Step 4: Ensemble the models and evaluate
    print("\nEvaluating ensemble model...")

    # Switch to validation mode
    dataset.set_mode('spatial')

    # Save models
    torch.save(vit_model.state_dict(), os.path.join(output_path, 'vit_model.pth'))
    torch.save(video_model.state_dict(), os.path.join(output_path, 'video_model.pth'))
    torch.save(adv_model.state_dict(), os.path.join(output_path, 'adversarial_model.pth'))

    # Evaluate on validation set
    vit_model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            outputs = vit_model(inputs)
            preds = torch.sigmoid(outputs).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels)

    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(output_path, 'roc_curve.png'))

    # Convert predictions to binary
    binary_preds = (all_preds > 0.5).astype(int)

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, binary_preds, target_names=['Fake', 'Real']))

    # Create confusion matrix
    cm = confusion_matrix(all_labels, binary_preds)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = [0, 1]
    plt.xticks(tick_marks, ['Fake', 'Real'])
    plt.yticks(tick_marks, ['Fake', 'Real'])

    # Add text annotations
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'confusion_matrix.png'))

    print(f"\nPhase 2 completed successfully! Results saved to {output_path}")
    return {
        'vit_model_path': os.path.join(output_path, 'vit_model.pth'),
        'video_model_path': os.path.join(output_path, 'video_model.pth'),
        'adversarial_model_path': os.path.join(output_path, 'adversarial_model.pth'),
        'roc_curve': os.path.join(output_path, 'roc_curve.png'),
        'confusion_matrix': os.path.join(output_path, 'confusion_matrix.png')
    }

# Install required packages
!pip install -q timm albumentations einops

# Execute Phase 2
if __name__ == "__main__":
    results = execute_phase2(real_faces_dir=real_faces_dir, fake_faces_dir=fake_faces_dir, output_path=output_path)
    print("Phase 2 model development and adversarial training complete!")

In [ ]:
!pip install -q gradio onnxruntime
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import gradio as gr
from einops import rearrange
import time
import shutil
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')


# Phase 3: Deployment & Real-World Testing
print("Starting Phase 3: Deployment & Real-World Testing")

# Define paths from previous phases
output_path = '/content/processed_data'
models_path = output_path

# Step 1: Load models from Phase 2
class ViTForDeepfake(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=pretrained)
        self.vit.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.vit.head.in_features, 1)
        )

    def forward(self, x):
        return self.vit(x)

class VideoTransformer(nn.Module):
    def __init__(self, num_frames=8, pretrained=False):
        super().__init__()
        self.backbone = timm.create_model('resnet18', pretrained=pretrained, features_only=True)

        # Get feature dimension
        dummy = torch.zeros(1, 3, 224, 224)
        features = self.backbone(dummy)
        feature_dim = features[-1].shape[1]

        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=feature_dim,
                nhead=8,
                dim_feedforward=2048,
                dropout=0.1,
                batch_first=True
            ),
            num_layers=2
        )

        self.classifier = nn.Sequential(
            nn.LayerNorm(feature_dim),
            nn.Linear(feature_dim, 1)
        )

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = rearrange(x, 'b t c h w -> (b t) c h w')

        features = self.backbone(x)
        x = features[-1]

        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(-1, x.size(1))

        x = rearrange(x, '(b t) c -> b t c', b=b, t=t)

        x = self.temporal_transformer(x)
        x = x.mean(dim=1)

        return self.classifier(x)

def load_models():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load spatial model
    spatial_model = ViTForDeepfake().to(device)
    spatial_model.load_state_dict(torch.load(os.path.join(models_path, 'vit_model.pth'),
                                            map_location=device))
    spatial_model.eval()

    # Load temporal model
    temporal_model = VideoTransformer().to(device)
    temporal_model.load_state_dict(torch.load(os.path.join(models_path, 'video_model.pth'),
                                             map_location=device))
    temporal_model.eval()

    return spatial_model, temporal_model, device

# Step 2: Define preprocessing functions
def get_transforms():
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2(),
    ])

def preprocess_image(image, transform):
    if isinstance(image, str):
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return transform(image=image)["image"]

def extract_frames(video_path, max_frames=32):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling to get evenly distributed frames
    if total_frames <= max_frames:
        frame_indices = range(total_frames)
    else:
        frame_indices = np.linspace(0, total_frames-1, max_frames, dtype=int)

    for i in frame_indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, frame = vidcap.read()
        if success:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

    vidcap.release()
    return frames

# Step 3: Define detection functions
def detect_image(image_path, model, device, transform):
    image = preprocess_image(image_path, transform)
    image = image.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()

    return prob

def detect_video(video_path, spatial_model, temporal_model, device, transform, temporal_window=8):
    # Extract frames
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return {"error": "No frames could be extracted from the video"}

    # Process frames
    processed_frames = [preprocess_image(frame, transform) for frame in frames]
    processed_frames = torch.stack(processed_frames).to(device)

    # Spatial analysis (frame by frame)
    spatial_probs = []
    for i in range(len(processed_frames)):
        with torch.no_grad():
            output = spatial_model(processed_frames[i].unsqueeze(0))
            prob = torch.sigmoid(output).item()
            spatial_probs.append(prob)

    # Temporal analysis (in windows)
    temporal_probs = []
    if len(frames) >= temporal_window:
        for i in range(0, len(frames) - temporal_window + 1, max(1, temporal_window // 2)):
            clip = processed_frames[i:i+temporal_window]
            if len(clip) < temporal_window:
                # Pad if necessary
                padding = temporal_window - len(clip)
                clip = torch.cat([clip, clip[-1].unsqueeze(0).repeat(padding, 1, 1, 1)])

            clip = clip.unsqueeze(0)  # Add batch dimension

            with torch.no_grad():
                output = temporal_model(clip)
                prob = torch.sigmoid(output).item()
                temporal_probs.append(prob)

    # Ensemble results
    if temporal_probs:
        final_prob = 0.6 * np.mean(spatial_probs) + 0.4 * np.mean(temporal_probs)
    else:
        final_prob = np.mean(spatial_probs)

    # Prepare visualization of frame-by-frame analysis
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(spatial_probs, '-o', label='Frame scores')
    ax.axhline(y=0.5, color='r', linestyle='--', label='Threshold')
    ax.set_title(f"Final score: {final_prob:.4f} ({'Real' if final_prob > 0.7 else 'Fake'})")
    ax.set_xlabel('Frame Number')
    ax.set_ylabel('Realness Score')
    ax.legend()

    # Save plot
    plot_path = '/tmp/detection_plot.png'
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

    return {
        "spatial_scores": spatial_probs,
        "temporal_scores": temporal_probs if temporal_probs else None,
        "final_score": final_prob,
        "prediction": "Real" if final_prob > 0.7 else "Fake",
        "confidence": abs(final_prob - 0.5) * 2,  # Scale to 0-1
        "plot": plot_path
    }

# Step 4: Model evaluation on challenging datasets
def evaluate_challenging_dataset(models, dataset_path, results_path):
    spatial_model, temporal_model, device = models
    transform = get_transforms()

    # Create results directory
    os.makedirs(results_path, exist_ok=True)

    # Find all video files
    video_files = glob(os.path.join(dataset_path, "**/*.mp4"), recursive=True)
    image_files = glob(os.path.join(dataset_path, "**/*.jpg"), recursive=True) + \
                 glob(os.path.join(dataset_path, "**/*.png"), recursive=True)

    results = {"videos": [], "images": []}

    # Process videos
    for video_path in tqdm(video_files, desc="Processing videos"):
        try:
            # Determine ground truth from path
            is_real = "real" in video_path.lower()
            label = 1 if is_real else 0

            # Run detection
            detection = detect_video(video_path, spatial_model, temporal_model, device, transform)

            # Store results
            results["videos"].append({
                "path": video_path,
                "label": label,
                "prediction": 1 if detection["final_score"] > 0.7 else 0,
                "score": detection["final_score"]
            })
        except Exception as e:
            print(f"Error processing {video_path}: {str(e)}")

    # Process images
    for image_path in tqdm(image_files, desc="Processing images"):
        try:
            # Determine ground truth from path
            is_real = "real" in image_path.lower()
            label = 1 if is_real else 0

            # Run detection
            score = detect_image(image_path, spatial_model, device, transform)

            # Store results
            results["images"].append({
                "path": image_path,
                "label": label,
                "prediction": 1 if score > 0.7 else 0,
                "score": score
            })
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

    # Calculate metrics
    def calculate_metrics(results_list):
        if not results_list:
            return None

        y_true = [r["label"] for r in results_list]
        y_scores = [r["score"] for r in results_list]
        y_pred = [r["prediction"] for r in results_list]

        # Calculate accuracy
        accuracy = sum(1 for t, p in zip(y_true, y_pred) if t == p) / len(y_true)

        # Calculate ROC
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr, tpr)

        # Calculate precision-recall
        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        pr_auc = auc(recall, precision)

        return {
            "accuracy": accuracy,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc,
            "fpr": fpr.tolist(),
            "tpr": tpr.tolist(),
            "precision": precision.tolist(),
            "recall": recall.tolist()
        }

    metrics = {
        "videos": calculate_metrics(results["videos"]),
        "images": calculate_metrics(results["images"])
    }

    # Save results
    with open(os.path.join(results_path, "evaluation_results.json"), "w") as f:
        import json
        json.dump({
            "metrics": metrics,
            "results": results
        }, f)

    # Plot ROC curves
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Video ROC
    if metrics["videos"]:
        ax1.plot(metrics["videos"]["fpr"], metrics["videos"]["tpr"],
                label=f'Video ROC (AUC = {metrics["videos"]["roc_auc"]:.3f})')
        ax1.plot([0, 1], [0, 1], 'k--')
        ax1.set_xlabel('False Positive Rate')
        ax1.set_ylabel('True Positive Rate')
        ax1.set_title('ROC Curve - Videos')
        ax1.legend()

    # Image ROC
    if metrics["images"]:
        ax2.plot(metrics["images"]["fpr"], metrics["images"]["tpr"],
                label=f'Image ROC (AUC = {metrics["images"]["roc_auc"]:.3f})')
        ax2.plot([0, 1], [0, 1], 'k--')
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('ROC Curve - Images')
        ax2.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(results_path, "roc_curves.png"))

    return metrics

# Step 5: Create Gradio interface for demo
def create_gradio_interface():
    # Load models
    spatial_model, temporal_model, device = load_models()
    transform = get_transforms()

    def process_image(image):
        # Process the image
        score = detect_image(image, spatial_model, device, transform)

        # Create result text
        prediction = "Real" if score > 0.7 else "Fake"
        confidence = abs(score - 0.5) * 2  # Scale to 0-1

        result_text = f"Prediction: {prediction}\nConfidence: {confidence:.2%}\nScore: {score:.4f}"

        return result_text

    def process_video(video):
        # Process the video
        results = detect_video(video, spatial_model, temporal_model, device, transform)

        # Create result text
        result_text = f"Prediction: {results['prediction']}\n"
        result_text += f"Confidence: {results['confidence']:.2%}\n"
        result_text += f"Score: {results['final_score']:.4f}"

        return result_text, results["plot"] if "plot" in results else None

    # Create Gradio interface
    with gr.Blocks(title="Deepfake Detection System") as demo:
        gr.Markdown("# Deepfake Detection System")
        gr.Markdown("Upload an image or video to detect if it's real or a deepfake.")

        with gr.Tabs():
            with gr.TabItem("Image Analysis"):
                with gr.Row():
                    with gr.Column():
                        image_input = gr.Image(type="pil", label="Upload Image")
                        image_button = gr.Button("Analyze Image")
                    with gr.Column():
                        image_output = gr.Textbox(label="Results")

                image_button.click(process_image, inputs=image_input, outputs=image_output)

            with gr.TabItem("Video Analysis"):
                with gr.Row():
                    with gr.Column():
                        video_input = gr.Video(label="Upload Video")
                        video_button = gr.Button("Analyze Video")
                    with gr.Column():
                        video_output = gr.Textbox(label="Results")
                        plot_output = gr.Image(label="Frame Analysis")

                video_button.click(process_video, inputs=video_input, outputs=[video_output, plot_output])

        gr.Markdown("## How it works")
        gr.Markdown("""
        This system uses advanced deep learning models to detect manipulated media:

        1. **Spatial Analysis**: Analyzes individual frames using Vision Transformers
        2. **Temporal Analysis**: Examines inconsistencies across video sequences
        3. **Ensemble Approach**: Combines multiple detection signals for robust results

        Higher scores (>0.7) indicate the media is likely real, while lower scores suggest manipulation.
        """)

    return demo

# Step 6: Model export for mobile/edge deployment

def export_models_for_deployment():
    # Load models
    spatial_model, temporal_model, device = load_models()

    # Create deployment directory
    deploy_dir = os.path.join(output_path, "deployment")
    os.makedirs(deploy_dir, exist_ok=True)

    # Export spatial model to ONNX with opset 14
    dummy_input = torch.randn(1, 3, 224, 224).to(device)

    try:
        torch.onnx.export(
            spatial_model,
            dummy_input,
            os.path.join(deploy_dir, "spatial_model.onnx"),
            export_params=True,
            opset_version=14,  # Changed from 11 to 14
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={
                'input': {0: 'batch_size'},
                'output': {0: 'batch_size'}
            }
        )
        print("✅ Spatial model exported successfully with opset 14")
    except Exception as e:
        print(f"❌ Failed to export spatial model: {str(e)}")
        return None
    '''
def export_models_for_deployment():
    # Load models
    spatial_model, temporal_model, device = load_models()

    # Create deployment directory
    deploy_dir = os.path.join(output_path, "deployment")
    os.makedirs(deploy_dir, exist_ok=True)

    # Export to ONNX format
    dummy_input = torch.randn(1, 3, 224, 224).to(device)
    torch.onnx.export(
        spatial_model,
        dummy_input,
        os.path.join(deploy_dir, "spatial_model.onnx"),
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'},
                     'output': {0: 'batch_size'}}
    )
    '''
    # Export metadata
    metadata = {
        "input_size": [224, 224],
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
        "threshold": 0.5
    }

    # Save metadata
    with open(os.path.join(deploy_dir, "model_metadata.json"), "w") as f:
        import json
        json.dump(metadata, f, indent=2)

    # Create a simple example script for inference
   # example_script =
    import numpy as np
    import cv2
    import onnxruntime

    def preprocess_image(image_path, size=(224, 224)):
        # Read and resize image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)

        # Normalize
        img = img.astype(np.float32) / 255.0
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = (img - mean) / std

        # HWC to CHW format
        img = img.transpose(2, 0, 1)

        # Add batch dimension
        img = np.expand_dims(img, 0)
        return img

    def run_inference(image_path, model_path):
        # Load image
        input_tensor = preprocess_image(image_path)

        # Create ONNX session
        session = onnxruntime.InferenceSession(model_path)

        # Run inference
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        result = session.run([output_name], {input_name: input_tensor})

        # Convert to probability
        score = 1 / (1 + np.exp(-result[0][0][0]))
        prediction = "Real" if score > 0.7 else "Fake"

        print(f"Prediction: {prediction}")
        print(f"Score: {score:.4f}")

        return {"prediction": prediction, "score": float(score)}

    # Example usage
    if __name__ == "__main__":
        import sys
        if len(sys.argv) < 3:
            print("Usage: python inference.py <image_path> <model_path>")
            sys.exit(1)

        image_path = sys.argv[1]
        model_path = sys.argv[2]
        result = run_inference(image_path, model_path)


    with open(os.path.join(deploy_dir, "inference.py"), "w") as f:
        f.write(example_script.strip())

    print(f"Models exported for deployment to {deploy_dir}")
    return deploy_dir

# Main execution function
def execute_phase3():
    print("Beginning Phase 3: Deployment & Real-World Testing")

    # Install required packages
    !pip install -q gradio onnxruntime onnx

    # Load models
    try:
        models = load_models()
        print("✅ Models loaded successfully")
    except Exception as e:
        print(f"❌ Failed to load models: {str(e)}")
        return

     # Create deployment package
    deploy_dir = None  # Initialize deploy_dir
    try:
        deploy_dir = export_models_for_deployment() # Assign value here
        print(f"✅ Deployment package created at {deploy_dir}")
    except Exception as e:
        print(f"❌ Failed to create deployment package: {str(e)}")

    # Create benchmark dataset
    benchmark_dir = os.path.join(output_path, "benchmark")
    os.makedirs(benchmark_dir, exist_ok=True)

    # Evaluate model on test data
    try:
        # Check if we have a test dataset
        test_dir = '/content/drive/MyDrive/Test'
        if os.path.exists(test_dir):
            print("Evaluating model on test dataset...")
            metrics = evaluate_challenging_dataset(
                models,
                test_dir,
                os.path.join(output_path, "evaluation_results")
            )
            print(f"✅ Model evaluation complete. Results saved to {os.path.join(output_path, 'evaluation_results')}")

            # Print summary metrics
            print("\n--- Performance Summary ---")
            if metrics["images"]:
                print(f"Image Detection: Accuracy={metrics['images']['accuracy']:.4f}, ROC-AUC={metrics['images']['roc_auc']:.4f}")
            if metrics["videos"]:
                print(f"Video Detection: Accuracy={metrics['videos']['accuracy']:.4f}, ROC-AUC={metrics['videos']['roc_auc']:.4f}")
        else:
            print("No test dataset found. Skipping evaluation.")
    except Exception as e:
        print(f"❌ Error during model evaluation: {str(e)}")

    # Create and launch demo interface
    try:
        print("Creating demo interface...")
        demo = create_gradio_interface()
        demo.launch(share=True)
        print("✅ Demo interface launched")
    except Exception as e:
        print(f"❌ Failed to launch demo: {str(e)}")

    print("\nPhase 3 completed successfully!")
    return {
        "deployment_dir": deploy_dir,
        "evaluation_results": os.path.join(output_path, "evaluation_results") if os.path.exists(test_dir) else None,
        "demo_interface": "Running in Gradio"
    }

# Execute Phase 3
if __name__ == "__main__":
    execute_phase3()